In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

display(HTML("<style>.container { width:100% !important; }</style>"))

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

import warnings
warnings.filterwarnings('default')

!pip install --upgrade scikit-learn


<IPython.core.display.Javascript object>

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x25f0f659840>
C:\Users\ouana\anaconda3\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\ouana\anaconda3\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\ouana\anaconda3\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


#### Nous disposons de 2 dataframe. Le second est équivalent au premier. La différence réside au niveau des colonnes selectionnées. Voici la listes des colonnes qui ont été enlevées dans df_model_2 :  

#### 'meta_user_review_negative', 'meta_user_review_mixed', 'meta_critic_review_mixed_perc', 'meta_user_review_mixed_perc', 'meta_user_review_positive_perc', 'meta_critic_review_negative_perc', 'meta_critic_review_positive_perc', 'meta_user_review_positive', 'meta_user_review_negative_perc'.

#### Dans un premier temps, nous allons effectuer nos test sur une petite partie de nos deux dataframe en selection les jeux présent uniquement sur les consoles de 7ème génération.

In [30]:
df_model_1 = pd.read_pickle("df_model_1.pkl")

In [31]:
df_model_1_platform_generation_7 = df_model_1.loc[df_model_1["platform_generation"] == 7]

In [42]:
# Séparation des variables numériques et catégorielle en prévision de l'entrainement du modèle.

numeric_features = [
 'release_month',
 'year',
 'meta_score',
 'meta_nb_user_critics',
 'meta_user_score',
 'meta_critic_review_positive_perc',
 'meta_critic_review_mixed_perc',
 'meta_user_review_positive_perc',
 'meta_user_review_positive',
 'meta_user_review_negative',
 'meta_user_review_mixed',
 'meta_user_review_mixed_perc',
 'meta_critic_review_negative_perc',
 'meta_user_review_negative_perc',
 'nb_platforms',
 'platform_generation',
 'genre_adventure',
 'genre_arcade',
 'genre_card-and-board-game',
 'genre_fighting',
 'genre_hack-and-slash-beat-em-up',
 'genre_indie',
 'genre_moba',
 'genre_music',
 'genre_pinball',
 'genre_platform',
 'genre_point-and-click',
 'genre_puzzle',
 'genre_quiz-trivia',
 'genre_racing',
 'genre_real-time-strategy-rts',
 'genre_role-playing-rpg',
 'genre_shooter',
 'genre_simulator',
 'genre_sport',
 'genre_strategy',
 'genre_tactical',
 'genre_turn-based-strategy-tbs',
 'genre_visual-novel',
 'mode_battle-royale',
 'mode_co-operative',
 'mode_mmo',
 'mode_multiplayer',
 'mode_single-player',
 'mode_split-screen',
 'perspective_auditory',
 'perspective_bird-view_isometric',
 'perspective_first-person',
 'perspective_side-view',
 'perspective_text',
 'perspective_third-person',
 'perspective_virtual-reality',
]

categorical_features = [
 'platform',
 'publisher',
 'developer',
 'platform_name',
 'platform_family', 
]


## 1) Pipeline Regression linéaire - PCA - Model 1 (generation platform 7)

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Création d'un pipeline afin d'assurer un processus clair et réutilisable par la suite

numeric_transformer = StandardScaler() # Initialisation d'un objet Standard Scaler
categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features), # Mise à l'échelle des variables numériques
        ('cat', categorical_transformer, categorical_features)]) # Application du one hot encoding sur les variable catégorielles

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('PCA', PCA(n_components = 5)),  # Selection des 5 composantes principales
    ('regression', LinearRegression()) # Selection du modèle
])

X = df_model_1_platform_generation_7[numeric_features + categorical_features]
y = df_model_1_platform_generation_7['total_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)



C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
from sklearn.metrics import mean_squared_error, r2_score

# Cette section est dédiée à l'évaluation des performances du modèle sur base de différentes métriques étudiées dans le cours

y_pred = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

mse = mean_squared_error(y_test, y_pred)
print('MSE: ', mse)

rmse = np.sqrt(mse)
print('RMSE: ', rmse)


R2:  0.16551192864675457
MSE:  39680093290409.16
RMSE:  6299213.704138728


#### Les performances du modèle sont faibles. Le R2 est seulement de 16.5% et l'erreur moyenne est supérieure à 6 millions

## 2) Pipeline Regression linéaire - PCA - Model 2 (generation platform 7)

In [47]:
df_model_2 = pd.read_pickle("df_model_2.pkl")

In [8]:
df_model_2_platform_generation_7 = df_model_2.loc[df_model_2["platform_generation"] == 7]

In [9]:
df_model_2.columns

Index(['total_sales', 'platform', 'release_month', 'year', 'publisher',
       'developer', 'meta_score', 'meta_nb_user_critics', 'meta_user_score',
       'game_collection', 'nb_platforms', 'genre_adventure', 'genre_arcade',
       'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
 

In [74]:
categorial_features_2 = [
 'platform_family',
 'platform_category',
 'platform_name',
 'game_collection',
 'developer',
"publisher",
"platform"]


In [75]:
numeric_features_2 = ['release_month', 'year',
       'meta_score', 'meta_nb_user_critics', 'meta_user_score',
       'nb_platforms', 'genre_adventure', 'genre_arcade',
       'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
       'perspective_third-person', 'perspective_virtual-reality',
        'platform_generation']


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Récupération du pipeline précédent en adaptant les variables à notre deuxième échantilon

numeric_transformer_2 = StandardScaler()
categorical_transformer_2 = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_2, numeric_features_2),
        ('cat', categorical_transformer_2, categorial_features_2)])

pipe_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_2),
    ('PCA', PCA(n_components = 5)), 
    ('regression', LinearRegression())
])

X_2 = df_model_2_platform_generation_7[numeric_features_2 + categorial_features_2]
y_2 = df_model_2_platform_generation_7['total_sales']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=0)

pipe_2.fit(X_train_2, y_train_2)

y_pred_2 = pipe_2.predict(X_test_2)

C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
from sklearn.metrics import mean_squared_error, r2_score

r2_2 = r2_score(y_test_2, y_pred_2)
print('R2: ', r2_2)

mse_2 = mean_squared_error(y_test_2, y_pred_2)
print('MSE: ', mse_2)

rmse_2 = np.sqrt(mse_2)
print('RMSE: ', rmse_2)

R2:  0.055988430340482065
MSE:  25439755610228.535
RMSE:  5043783.858397239


#### Nous pouvons observer une faible performance de la part de notre deuxième modele basé sur le df_model_2. 

#### Nous allons désormais effectuer nos test sur l'entireté de notre df_model_1 contenant toutes les colonnes et nous ferons de même pour le df_model_2.

### Test modele 1 complet

In [133]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('PCA', PCA(n_components = 5)), 
    ('regression', LinearRegression())
])

X = df_model_1[numeric_features + categorical_features]
y = df_model_1['total_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)


C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [132]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

mse = mean_squared_error(y_test, y_pred)
print('MSE: ', mse)

rmse = np.sqrt(mse)
print('RMSE: ', rmse)

R2:  -4.097988426997127
MSE:  41081218785911.66
RMSE:  6409463.221355721


#### Les performances du modèle avec ce dataframe sont mauvaises. Nous verrons la différence avec le second df dans le prochain code.

In [146]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

numeric_transformer_2 = StandardScaler()
categorical_transformer_2 = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_2, numeric_features_2),
        ('cat', categorical_transformer_2, categorial_features_2)])

pipe_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_2),
    ('PCA', PCA(n_components = 5)), 
    ('regression', LinearRegression())
])

X_2 = df_model_2[numeric_features_2 + categorial_features_2]
y_2 = df_model_2['total_sales']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

pipe_2.fit(X_train_2, y_train_2)

y_pred_2 = pipe_2.predict(X_test_2)

C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [147]:
df_model_2["total_sales"].mean()

774687.4642243846

In [148]:
from sklearn.metrics import mean_squared_error, r2_score

r2_2 = r2_score(y_test_2, y_pred_2)
print('R2: ', r2_2)

mse_2 = mean_squared_error(y_test_2, y_pred_2)
print('MSE: ', mse_2)

rmse_2 = np.sqrt(mse_2)
print('RMSE: ', rmse_2)

R2:  0.06904348238791891
MSE:  6177612631681.766
RMSE:  2485480.362360919


#### Les résultats d'évaluation du modèle ne sont pas acceptables pour aucuns des 2 dataframe

#### Ce modèle n'est pas adéquate dans notre cas. Une forêt aléatoire est un type modèle peut-être plus adapté à notre problématique

# Nous allons nous pencher sur un autre modele et allons l'appliquer à nos 2 dataframe.
# Le Random Forest sans PCA

In [181]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import shap
import numpy as np

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('regressor', RandomForestRegressor(random_state = 42))
])

X = df_model_1[numeric_features + categorical_features]
y = df_model_1['total_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)


from sklearn.metrics import mean_squared_error, r2_score

y_pred = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

mse = mean_squared_error(y_test, y_pred)
print('MSE: ', mse)

rmse = np.sqrt(mse)
print('RMSE: ', rmse)



`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


R2:  0.2201594368546862
MSE:  31692142304193.68
RMSE:  5629577.453432334


## Les résultats sur le premier df ne sont pas concluant mais restent supérieur à ceux retourner par le modele de Regression Linéaire.

## Nous allons effectuer le même test avec le df2 et observer les résultats

In [150]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

numeric_transformer_2 = StandardScaler()
categorical_transformer_2 = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_2, numeric_features_2),
        ('cat', categorical_transformer_2, categorial_features_2)])

pipe_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_2), 
    ('regressor', RandomForestRegressor(random_state = 42))
])

X_2 = df_model_2[numeric_features_2 + categorial_features_2]
y_2 = df_model_2['total_sales']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

pipe_2.fit(X_train_2, y_train_2)

y_pred_2 = pipe_2.predict(X_test_2)


C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [151]:
from sklearn.metrics import mean_squared_error, r2_score

r2_2 = r2_score(y_test_2, y_pred_2)
print('R2: ', r2_2)

mse_2 = mean_squared_error(y_test_2, y_pred_2)
print('MSE: ', mse_2)

rmse_2 = np.sqrt(mse_2)
print('RMSE: ', rmse_2)

R2:  0.5205006004498762
MSE:  3181847370425.6987
RMSE:  1783773.3517534393


#### Les résultats sont acceptables pour ce dernier test effectué sur le deuxième dataframe comparativement à tous les autres modèles, malgré l'application de la PCA ou non. Cependant, les performances de ce dernier modèle ne sont pas aussi satisfaisant que ce qui était attendu. Le R2 est de 52% et la RMSE est de 1 783 773.

#### En ce qui concerne la procédure, nous avons créer un pipeline dès le début afin de le réutiliser par la suite tout en adaptant les variables aux nouveau modèles testés. Cela nous a permis de gagner du temps, et d'éviter les erreurs d'inattention qui auraient pu fausser les résultats de nos modèles.

### Désormais, nous allons refaire les mêmes tests que nous venons d'éffectuer mais nous allons le faire sur nos deux dataframe (Platform_generation_7) avec toutes les valeurs SAUF les lignes contenant un "All". df_model_1 contiendra désormais des lignes/colonnes dans lesquelels nous ne pourront trouver aucun "All'.

In [37]:
df_model_1 = df_model_1[~(df_model_1 == 'All').any(axis=1)]
len(a)

8873

In [41]:
df_model_1_platform_generation_7 = df_model_1.loc[df_model_1["platform_generation"] == 7]
df_model_1_platform_generation_7

total_sales platform  release_month  year               publisher  \
0       82900000.0      Wii             11  2006                Nintendo   
4       40000000.0       PC              8  2012                   Valve   
5       37380000.0      Wii              4  2008                Nintendo   
8       33140000.0      Wii              7  2009                Nintendo   
10      30800000.0       DS              5  2006                Nintendo   
...            ...      ...            ...   ...                     ...   
19940      10000.0     X360              3  2007             Codemasters   
19955      10000.0       DS              6  2006            D3 Publisher   
19957      10000.0       PC              5  2009  Focus Home Interactive   
19973      10000.0       PC              7  2011                 Kalypso   
19976      10000.0       PC              7  2009                 Nobilis   

                   developer  meta_score  meta_nb_user_critics  \
0               Nintendo EAD        76.0                 543.0   
4          Valve Corporation        83.0                5143.0   
5               Nintendo EAD        82.0                1194.0   
8               Nintendo EAD        80.0                 350.0   
10              Nintendo EAD        89.0                 838.0   
...                      ...         ...                   ...   
19940            Codemasters        71.0                  10.0   
19955  Sensory Sweep Studios        52.0                   9.0   
19957              Frogwares        73.0                  67.0   
19973           Catnip Games        71.0                   0.0   
19976             Frozenbyte        80.0                 614.0   

       meta_user_score  meta_critic_review_positive_perc  ...  \
0                 81.0                              61.0  ...   
4                 72.0                              89.0  ...   
5                 85.0                              89.0  ...   
8                 83.0                              82.0  ...   
10                86.0                              95.0  ...   
...                ...                               ...  ...   
19940             84.0                              29.0  ...   
19955             72.0                               0.0  ...   
19957             75.0                              50.0  ...   
19973              0.0                              57.0  ...   
19976             81.0                              86.0  ...   

       perspective_bird-view_isometric  perspective_first-person  \
0                                False                      True   
4                                False                      True   
5                                False                     False   
8                                False                      True   
10                               False                     False   
...                                ...                       ...   
19940                            False                     False   
19955                            False                     False   
19957                            False                      True   
19973                            False                     False   
19976                            False                     False   

       perspective_side-view  perspective_text  perspective_third-person  \
0                      False             False                      True   
4                      False             False                     False   
5                      False             False                      True   
8                      False             False                      True   
10                      True             False                     False   
...                      ...               ...                       ...   
19940                  False             False                      True   
19955                  False             False                     False   
19957              

In [43]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Création d'un pipeline afin d'assurer un processus clair et réutilisable par la suite

numeric_transformer = StandardScaler() # Initialisation d'un objet Standard Scaler
categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features), # Mise à l'échelle des variables numériques
        ('cat', categorical_transformer, categorical_features)]) # Application du one hot encoding sur les variable catégorielles

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('PCA', PCA(n_components = 5)),  # Selection des 5 composantes principales
    ('regression', LinearRegression()) # Selection du modèle
])

X = df_model_1_platform_generation_7[numeric_features + categorical_features]
y = df_model_1_platform_generation_7['total_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)



C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [44]:
from sklearn.metrics import mean_squared_error, r2_score

# Cette section est dédiée à l'évaluation des performances du modèle sur base de différentes métriques étudiées dans le cours

y_pred = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

mse = mean_squared_error(y_test, y_pred)
print('MSE: ', mse)

rmse = np.sqrt(mse)
print('RMSE: ', rmse)

R2:  0.20282305489398755
MSE:  6099840395054.584
RMSE:  2469785.495757594


### Nous observons une augmentation de 4% du R2 après avoir supprimé les lignes contenant un "All" comparé au modèle précédent (le premier modèle que nous testé). Nous allons faire de même pour le dataframe en entier : df_model_1 entier

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('PCA', PCA(n_components = 5)), 
    ('regression', LinearRegression())
])

X = df_model_1[numeric_features + categorical_features]
y = df_model_1['total_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [46]:
from sklearn.metrics import mean_squared_error, r2_score

# Cette section est dédiée à l'évaluation des performances du modèle sur base de différentes métriques étudiées dans le cours

y_pred = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

mse = mean_squared_error(y_test, y_pred)
print('MSE: ', mse)

rmse = np.sqrt(mse)
print('RMSE: ', rmse)

R2:  0.1254731142760922
MSE:  3557410736319.1416
RMSE:  1886109.9480992993


### Le R2 est passé de -4% à 12%. Malgré un faible résultat, nous pourrons observer que la suppression des lignes contenant "All" permettent une meileur performance de nos modèles. Par la suite, nous effectueront nos tests sur le df_model_2.

In [72]:
df_model_2 = df_model_2[~(df_model_2 == 'All').any(axis=1)]
df_model_2

total_sales platform  release_month  year                 publisher  \
0       82900000.0      Wii             11  2006                  Nintendo   
1       52000000.0       NS              4  2017                  Nintendo   
2       41590000.0       NS              3  2020                  Nintendo   
3       40240000.0      NES              9  1985                  Nintendo   
4       40000000.0       PC              8  2012                     Valve   
...            ...      ...            ...   ...                       ...   
19977      10000.0     X360              5  2014    Focus Home Interactive   
19979      10000.0       PC             10  2015              Nordic Games   
19980      10000.0       PC              5  2009              Namco Bandai   
19982      10000.0      GBA              9  2006                       THQ   
19983      10000.0       GC              5  2002  Conspiracy Entertainment   

                developer  meta_score  meta_nb_user_critics  meta_user_score  \
0            Nintendo EAD   76.000000                 543.0        81.000000   
1            Nintendo EPD   92.000000                2701.0        86.000000   
2                Nintendo   90.000000                6586.0        56.000000   
3            Nintendo EAD   73.330000                1087.0        83.904607   
4       Valve Corporation   83.000000                5143.0        72.000000   
...                   ...         ...                   ...              ...   
19977             Spiders   57.363636                  13.0        62.969737   
19979     Rainbow Studios   20.000000                  20.0        50.000000   
19980  Namco Tales Studio   48.666667                  56.0        69.980000   
19982     Barking Lizards   70.170000                   6.0        63.850000   
19983             Warthog   63.000000                   0.0         0.000000   

                        game_collection  ...  perspective_bird-view_isometric  \
0                            Wii Sports  ...                            False   
1                            Mario Kart  ...                            False   
2                       Animal Crossing  ...                             True   
3                     Super Mario Bros.  ...                            False   
4                        Counter-Strike  ...                            False   
...                                 ...  ...                              ...   
19977                    bound by flame  ...                            False   
19979                                MX  ...                            False   
19980  brain exercise with dr kawashima  ...                            False   
19982            bratz forever diamondz  ...                            False   
19983                rally championship  ...                            False   

       perspective_first-person  perspective_side-view  perspective_text  \
0                          True                  False             False   
1                         False                  False             False   
2                         False                  False             False   
3                         False                   True             False   
4                          True                  False             False   
...                         ...                    ...               ...   
19977                     False                  False             False   
19979                      True                  False             False   
19980                     False                  False             False   
19982                     False                  False             False   
19983                      True                  False             False   

       perspective_third-person  perspective_virtual-reality  \
0                          True                        False   
1                          True                        False   
2                         False

In [73]:
df_model_2_platform_generation_7 = df_model_2.loc[df_model_2["platform_generation"] == 7]

In [70]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Récupération du pipeline précédent en adaptant les variables à notre deuxième échantilon

numeric_transformer_2 = StandardScaler()
categorical_transformer_2 = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_2, numeric_features_2),
        ('cat', categorical_transformer_2, categorial_features_2)])

pipe_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_2),
    ('PCA', PCA(n_components = 5)), 
    ('regression', LinearRegression())
])

X_2 = df_model_2_platform_generation_7[numeric_features_2 + categorial_features_2]
y_2 = df_model_2_platform_generation_7['total_sales']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=0)

pipe_2.fit(X_train_2, y_train_2)

y_pred_2 = pipe_2.predict(X_test_2)

r2_2 = r2_score(y_test_2, y_pred_2)
print('R2: ', r2_2)

mse_2 = mean_squared_error(y_test_2, y_pred_2)
print('MSE: ', mse_2)

rmse_2 = np.sqrt(mse_2)
print('RMSE: ', rmse_2)

C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


R2:  0.07758473151366652
MSE:  4048218700089.419
RMSE:  2012018.5635548742


### Le R2 a augmenté seulement de 2% comparé au test que nous avons effectué précédemment avec le même filtre sur la platform_generation_7 du df_modele_2

In [76]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Récupération du pipeline précédent en adaptant les variables à notre deuxième échantilon

numeric_transformer_2 = StandardScaler()
categorical_transformer_2 = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_2, numeric_features_2),
        ('cat', categorical_transformer_2, categorial_features_2)])

pipe_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_2),
    ('PCA', PCA(n_components = 5)), 
    ('regression', LinearRegression())
])

X_2 = df_model_2[numeric_features_2 + categorial_features_2]
y_2 = df_model_2['total_sales']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=0)

pipe_2.fit(X_train_2, y_train_2)

y_pred_2 = pipe_2.predict(X_test_2)

r2_2 = r2_score(y_test_2, y_pred_2)
print('R2: ', r2_2)

mse_2 = mean_squared_error(y_test_2, y_pred_2)
print('MSE: ', mse_2)

rmse_2 = np.sqrt(mse_2)
print('RMSE: ', rmse_2)

C:\Users\ouana\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


R2:  0.02933662422946859
MSE:  2975566564179.1943
RMSE:  1724983.0619977678


### Dans ce test, les performances sont encore moins bonnes que dans le modèle précédent.

### Le prochain test sera effectué avec un modele de Random Forest sur lequel nous appliquerons la PCA. Le Random Forest est un modèle capable (à l'inverse de la regression linéaire) de gérer une grande quantité de variables. 

#### Test Model 1 sans "All" - Random Forest 

In [77]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import shap
import numpy as np

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('regressor', RandomForestRegressor(random_state = 42))
])

X = df_model_1[numeric_features + categorical_features]
y = df_model_1['total_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)


from sklearn.metrics import mean_squared_error, r2_score

y_pred = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

mse = mean_squared_error(y_test, y_pred)
print('MSE: ', mse)

rmse = np.sqrt(mse)
print('RMSE: ', rmse)

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


R2:  0.4717495736885653
MSE:  4106365934728.5713
RMSE:  2026417.0189594666


#### Test Model 1 sans "All" - Random Forest 

In [78]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

numeric_transformer_2 = StandardScaler()
categorical_transformer_2 = OneHotEncoder(sparse=False, handle_unknown = "ignore")

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_2, numeric_features_2),
        ('cat', categorical_transformer_2, categorial_features_2)])

pipe_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_2), 
    ('regressor', RandomForestRegressor(random_state = 42))
])

X_2 = df_model_2[numeric_features_2 + categorial_features_2]
y_2 = df_model_2['total_sales']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

pipe_2.fit(X_train_2, y_train_2)

y_pred_2 = pipe_2.predict(X_test_2)

from sklearn.metrics import mean_squared_error, r2_score

r2_2 = r2_score(y_test_2, y_pred_2)
print('R2: ', r2_2)

mse_2 = mean_squared_error(y_test_2, y_pred_2)
print('MSE: ', mse_2)

rmse_2 = np.sqrt(mse_2)
print('RMSE: ', rmse_2)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


R2:  0.5084248178239049
MSE:  2506513049532.041
RMSE:  1583197.0974998788
